# University Admission Prediction

In [26]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("datasets/adm_data.csv")
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [7]:
X = df.drop(["Chance of Admit "], axis=1)
y = pd.DataFrame(df["Chance of Admit "]).copy()

In [10]:
feature_names = X.columns

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.20, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(320, 8) (80, 8) (320, 1) (80, 1)


In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

## Train a Linear Regression Model

In [31]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor

In [35]:
regressors = [LinearRegression(),
             RandomForestRegressor(),
             KNeighborsRegressor()]

for i in regressors:
    regressor = i
    regressor.fit(X_train, y_train())
    
    print(regressor)
    print(f"Training R2 = {regressor.score(X_train, y_train):.3f}")
    print(f"Testing R2 = {regressor.score(X_test, y_test):.3f}\n")
    


# mlr = LinearRegression()
# mlr.fit(X_train, y_train)

LinearRegression()
Training R2 = 0.816
Testing R2 = 0.817

RandomForestRegressor()
Training R2 = 0.973
Testing R2 = 0.823

KNeighborsRegressor()
Training R2 = 0.865
Testing R2 = 0.809



In [39]:
r1 = LinearRegression()
r2 = RandomForestRegressor()
r3 = KNeighborsRegressor()

vr = VotingRegressor([('lr', r1), ('rf', r2), ('r3', r3)])
vr.fit(X_train, y_train.values.ravel()).score(X_test, y_test)

0.8334087815156155